# Import Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
#Loading data from a CSV file
df = pd.read_csv('/kaggle/input/stock-price-prediction-of-apple-inc/apple_share_price.csv')

In [ ]:
df.isna().sum()

In [ ]:
df.head()

In [ ]:
df.shape

**df.reset_index(): This method is used to reset the index of a DataFrame. When you reset the index, the current index becomes a new column, and a default integer index is assigned to the DataFrame. This operation is often used to move the existing index (possibly a datetime index) to a regular column.**

 **['Close']: After resetting the index, the code selects the 'Close' column from the resulting DataFrame. This is done by using the square brackets (['Close']) to access the column with the label 'Close'.**

In [ ]:
df1=df.reset_index()['Close'];df1

In [ ]:
plt.plot(df1)

# Normalize Data

**Normalize the data to ensure that all features have the same scale. This is important for the LSTM to learn effectively.**

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
df['Close'] = scaler.fit_transform(np.array(df['Close']).reshape(-1, 1))


# Create Sequences for LSTM

**Create input sequences for the LSTM model. This involves defining a window of past data that the model will use for prediction.**

In [ ]:
def create_sequences(data,seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        sequences.append(seq)
    return np.array(sequences)

seq_length = 10  # Adjust as needed
X = create_sequences(df['Close'], seq_length)

# Split Data into Training and Testing Sets:

In [ ]:
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size
train, test = X[0:train_size], X[train_size:len(X)]

# Model

In [ ]:
from tensorflow.keras.layers import Dropout

model = Sequential()

# Add the first LSTM layer with return_sequences=True
model.add(LSTM(units=100, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(Dropout(0.2))  # Add dropout for regularization

# Add a second LSTM layer with return_sequences=True
model.add(LSTM(units=100, return_sequences=True))
model.add(Dropout(0.2))

# Add a third LSTM layer with return_sequences=True
model.add(LSTM(units=100, return_sequences=True))
model.add(Dropout(0.2))

# Add a fourth LSTM layer without return_sequences (default is False)
model.add(LSTM(units=100))
model.add(Dropout(0.2))

# Add a Dense layer
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# LSTM Model Architecture
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='lstm_model.png', show_shapes=True, show_layer_names=True)


In [ ]:
target_data = df['Close'][seq_length:seq_length+train_size]
model.fit(train, target_data, epochs=10, batch_size=32)


In [ ]:
target_data = df['Close'][seq_length:seq_length+train_size]
historyy = model.fit(train, target_data, epochs=10, batch_size=32,validation_split=0.2)


In [ ]:
history_df = pd.DataFrame(historyy.history)
history_df.loc[:, ['loss', 'val_loss']].plot();

In [ ]:
# Make Predictions
predictions = model.predict(test)
predictions = scaler.inverse_transform(predictions)

# Plot Results

In [ ]:
train = df.iloc[:train_size, :]
test = df.iloc[train_size+seq_length:, :]

test['Predictions'] = predictions
plt.figure(figsize=(16,8))
plt.title('Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Close Price (USD)')
plt.plot(train['Close'])
plt.plot(test[['Close', 'Predictions']])
plt.legend(['Train', 'Test', 'Predictions'])
plt.show()

In [ ]:
#Prediction Error Distribution
error = test['Close'] - test['Predictions']
plt.figure(figsize=(10,6))
plt.hist(error, bins=30)
plt.title('Prediction Error Distribution')
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Feature Importance (Input Weights)
weights = model.layers[0].get_weights()[0]
plt.figure(figsize=(12,6))
plt.bar(range(len(weights)), weights[:, 0])
plt.title('Input Weights of LSTM Layer')
plt.xlabel('Input Feature Index')
plt.ylabel('Weight Value')
plt.show()


In [ ]:
#Scatter Plot of Actual vs. Predicted Prices
plt.figure(figsize=(10,6))
plt.scatter(test['Close'], test['Predictions'])
plt.title('Scatter Plot of Actual vs. Predicted Prices')
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices')
plt.show()


In [ ]:
# Residual Plot
plt.figure(figsize=(10,6))
plt.scatter(test['Date'], error)
plt.title('Residual Plot')
plt.xlabel('Date')
plt.ylabel('Prediction Error')
plt.show()


In [ ]:
# Rolling Mean of Prediction Errors
rolling_mean = error.rolling(window=10).mean()
plt.figure(figsize=(10,6))
plt.plot(test['Date'], rolling_mean)
plt.title('Rolling Mean of Prediction Errors')
plt.xlabel('Date')
plt.ylabel('Rolling Mean of Prediction Error')
plt.show()


In [ ]:
# Distribution of Training and Test Sets
plt.figure(figsize=(12,6))
plt.hist(train['Close'], bins=30, alpha=0.5, label='Training Set')
plt.hist(test['Close'], bins=30, alpha=0.5, label='Test Set')
plt.title('Distribution of Stock Prices in Training and Test Sets')
plt.xlabel('Close Price (USD)')
plt.ylabel('Frequency')
plt.legend()
plt.show()


# THANK YOU